In [6]:
from pyinjective.async_client import AsyncClient
from pyinjective.core.network import Network

import pandas as pd 


network = Network.local()
client = AsyncClient(network)


async def sub_order(client, account_address) -> None:
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]


async def order_list(client, subaccount_id,market_id) -> None:
    orders = await client.fetch_subaccount_orders_list(subaccount_id=subaccount_id, market_id=market_id)
    return orders['orders']


async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(client, account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(client, i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    ## sort by quantity_in_$
    df = df.sort_values(by="quantity_in_$", ascending=False)
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    
    return df

In [7]:
from functions import marketId_to_ticker
from IPython.display import display
import pandas as pd

tickers = ["iMSTR/USDT PERP", "iNVDA/USDT PERP", "iCOIN/USDT PERP", "iTSLA/USDT PERP", "iHOOD/USDT PERP", "iMCD/USDT PERP", "iGOOGL/USDT PERP", "TRADFI/USDT PERP", "TTI/USDT PERP"]

address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

async def main():

    combined_df = pd.DataFrame()

    for ticker in tickers:
        market_id = await marketId_to_ticker(ticker)
        if market_id:
            print(f"Fetching orders for {ticker}...")
            df = await subfinder(account_address=address, market_id=market_id[0])
            
            # df = df.iloc[:1]
            
            df['ticker'] = ticker
            
            display(df)
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            column_to_drop = ["orderSide","price", "quantity_in_$"]
            combined_df.drop(columns=column_to_drop, inplace=True)

    combined_df.to_csv(f"orders_{address}.csv", index=False)
    print(f"Combined CSV saved as orders_{address}.csv")

await main()

Fetching orders for iMSTR/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
19,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000046,301.18,9336.58000,iMSTR/USDT PERP
18,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000046,300.41,9312.71000,iMSTR/USDT PERP
17,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,320.62,1627.14650,iMSTR/USDT PERP
16,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,318.28,1615.27100,iMSTR/USDT PERP
15,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,317.41,1610.85575,iMSTR/USDT PERP
14,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,317.12,1609.38400,iMSTR/USDT PERP
13,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,316.82,1607.86150,iMSTR/USDT PERP
12,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,313.61,1591.57075,iMSTR/USDT PERP
11,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,310.40,1575.28000,iMSTR/USDT PERP
10,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000003f,310.11,1573.80825,iMSTR/USDT PERP


Fetching orders for iNVDA/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
20,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000047,108.09,204038.03412,iNVDA/USDT PERP
21,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000047,108.39,9755.10000,iNVDA/USDT PERP
19,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000047,108.30,9747.00000,iNVDA/USDT PERP
18,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,120.42,437.72670,iNVDA/USDT PERP
17,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,119.55,434.56425,iNVDA/USDT PERP
16,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,119.22,433.36470,iNVDA/USDT PERP
15,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,119.11,432.96485,iNVDA/USDT PERP
14,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,119.00,432.56500,iNVDA/USDT PERP
13,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,117.79,428.16665,iNVDA/USDT PERP
12,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000042,116.58,423.76830,iNVDA/USDT PERP


Fetching orders for iCOIN/USDT PERP...


...

Fetching orders for TRADFI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
1,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,555.84,182545.63776,TRADFI/USDT PERP
2,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,562.09,10117.62000,TRADFI/USDT PERP
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004d,557.49,10034.82000,TRADFI/USDT PERP


Fetching orders for TTI/USDT PERP...


,orderSide,subaccountId,price,quantity_in_$,ticker
21,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004e,469.08,9850.6800,TTI/USDT PERP
20,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000004e,468.95,9847.9500,TTI/USDT PERP
19,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,514.97,437.7245,TTI/USDT PERP
18,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,511.21,434.5285,TTI/USDT PERP
17,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,509.81,433.3385,TTI/USDT PERP
16,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,509.34,432.9390,TTI/USDT PERP
15,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,508.87,432.5395,TTI/USDT PERP
14,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,503.71,428.1535,TTI/USDT PERP
13,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,498.55,423.7675,TTI/USDT PERP
12,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000044,498.08,423.3680,TTI/USDT PERP


Combined CSV saved as orders_inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z.csv
